In [6]:
# ----- Extraction PDF et découpage du texte -----

from pypdf import PdfReader

# Fonction pour extraire le texte de chaque page d’un PDF
def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    return " ".join(page.extract_text() or "" for page in reader.pages)

# Fonction pour découper le texte en morceaux de taille raisonnable
def split_text(text, max_length=500):
    sentences = text.split(". ")
    chunks, chunk = [], ""
    for sentence in sentences:
        if len(chunk) + len(sentence) < max_length:
            chunk += sentence + ". "
        else:
            chunks.append(chunk.strip())
            chunk = sentence + ". "
    if chunk:
        chunks.append(chunk.strip())
    return chunks

# Nettoie les caractères indésirables dans un morceau de texte
def clean_chunk(chunk):
    return chunk.replace("\xa0", " ").replace("\n", " ").strip()

In [7]:
# 📄 Lecture du PDF et préparation des chunks
pdf_path = "data/loups_garous.pdf"
raw_text = extract_text_from_pdf(pdf_path)  # Extraction du texte brut
chunks = split_text(raw_text)               # Découpage en morceaux exploitables
print(f"✅ {len(chunks)} morceaux extraits.")  # Affichage du nombre de morceaux extraits

✅ 40 morceaux extraits.


In [8]:
# 🔡 Embedding des textes + FAISS
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Chargement du modèle d'embedding multilingue
embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus_embeddings = embedder.encode(chunks, convert_to_numpy=True)  # Vectorisation des chunks

# Création de l'index FAISS pour la recherche rapide par similarité
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)  # Ajout des embeddings à l'index

In [10]:
# 🤖 Chargement du modèle de génération de texte
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen3-0.6B"  # Modèle léger pour des réponses rapides
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16,      # Utilisation de la mémoire optimisée
    device_map="auto"               # Utilise automatiquement le GPU si dispo
)

In [11]:
import gradio as gr

# Fonction principale : génère une réponse à partir d’une question
def generate_response(question):
    # Recherche des morceaux de texte les plus pertinents (top 5)
    q_embedding = embedder.encode([question], convert_to_numpy=True)
    D, I = index.search(q_embedding, k=5)
    
    # Nettoyage et assemblage du contexte extrait
    top_chunks = [clean_chunk(chunks[i]) for i in I[0]]
    context = " ".join(top_chunks)
    
    # Prompt envoyé au modèle : consignes + contexte + question
    prompt = f"""
Tu es un expert des règles de jeux de société. Ci-dessous se trouve un extrait du manuel de règles.

Contexte extrait du manuel :
{context}

Question posée par un joueur :
{question}

Réponds précisément et clairement en te basant uniquement sur ce contexte. Si la réponse n’est pas dans le texte, indique-le poliment. Sois pédagogique et clair.
"""
    # Encodage du prompt pour le modèle
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    # Génération de la réponse sans apprentissage (inférence)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,      # Limite la longueur de la réponse
            temperature=0.7,         # Créativité modérée
            top_p=0.9,               # Nucleus sampling
            do_sample=True,          # Génération aléatoire
            num_beams=1              # Pas de recherche faisceau (plus rapide)
        )
    
    # Décodage du texte généré + récupération des chunks utilisés
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    used_chunks = "\n\n".join(f"[Chunk {i+1}]\n{chunks[i]}" for i in I[0])
    
    return response.strip(), used_chunks

# 🔧 Interface Gradio adaptée à la consultation de règles
iface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Exemple : Que se passe-t-il si deux joueurs ont le même nombre de votes ?", label="Question sur une règle du jeu"),
    outputs=[
        gr.Textbox(label="Réponse claire à votre question"),                    # Réponse générée
        gr.Textbox(label="Extraits du manuel utilisés (pour transparence)")    # Chunks utilisés
    ],
    title="Assistant Règles - Jeu de Société",
    description="Posez une question sur une règle que vous ne comprenez pas, et l'assistant vous explique simplement à partir du manuel."
)

iface.launch()  # Lancement de l'application Gradio

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
